<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/master/LinkWithTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation de la library *python-dotenv*

In [ ]:
!pip install python-dotenv

Importation des libraries

In [ ]:
import tweepy
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import requests
from IPython.display import JSON

Liaison au Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/PFE/"

Mounted at /content/drive


Récupération dans ***data*** des donnés nécessaires de tableauSites.csv

In [ ]:
data = pd.read_csv(DATA_PATH+"tableauSites.csv", encoding='utf-8').drop(['Unnamed: 0', 'id'], axis=1).values.tolist()

Vérification

In [ ]:
data

[['France Soir', 2, 'france_soir'],
 ['Régénère (Thierry Casasnovas)', 2, 'ThierryRegenere'],
 ['Silvano Trotta', 2, 'silvano_trotta'],
 ['Ema Krusi', 2, 'emakrusi'],
 ['Salim Laïbi (Le Libre Penseur)', 2, 'llp_le_vrai'],
 ['https://covidinfos.net/', 2, 'covid_infos'],
 ['Carlo Alberto Brusa', 3, 'cab2626'],
 ['La Mentable', 1, 'lamentableFR'],
 ['Elishean-Aufeminin.com', 2, 'Elishean'],
 ['NoSignalFound', 2, 'DrMo7oG'],
 ['Les chroniques du Yéti', 2, 'yetiblog'],
 ['Le Politique', 3, 'lepolitique_fr'],
 ['Les Répliques', 3, 'les_repliques'],
 ['Damocles', 3, 'damocles_fr'],
 ['Jnewsfoot', 1, 'jnewsfoot'],
 ["Le Journal de L'Elysée", 1, 'JournalElysee'],
 ['20 secondes', 1, '20secondesfr'],
 ['Golf Investigation', 1, 'Golfinvestigat1'],
 ['Institut International Indépendant pour la Promotion de la Recherche Scientifique',
  1,
  'iiiprs'],
 ['Vox', 4, 'voxdotcom'],
 ["L'Usine Nouvelle", 4, 'usinenouvelle'],
 ['Les Observateurs de France 24', 4, 'Observateurs'],
 ['Radio Télévision Suis

Récupération des clés pour se connecter au compte Twitter Developers et utiliser son API

In [ ]:
API_TOKEN = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['Bearer Token'][0]
API_KEY = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key'][0]
API_SECRET = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key Secret'][0]

Authentifiation au compte Twitter Developers et récupération de l'API tweepy

In [ ]:
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

Fonction pour récupérer les tweets d'un utilisateurs, basé sur une solution disponible sur :

https://towardsdatascience.com/collect-data-from-twitter-a-step-by-step-implementation-using-tweepy-7526fff2cb31

In [ ]:
from tweepy import Cursor # Used to perform pagination

# Function creating final list
def get_tweets_from_user(twitter_user_name, liability, api, page_limit=15, count_tweet=200):
    """
    @params:
        - twitter_user_name: the twitter username of a user (company, etc.)
        - page_limit: the total number of pages (max=16)
        - count_tweet: maximum number to be retrieved from a page
        
    @return
        - all the tweets from the user twitter_user_name
    """
    liability_label = ['Site parodique', 'Site non fiable', 'Site à la fiabilité douteuse', 'Site réputé fiable']
    all_tweets = []
    try:
      print("Récupération des tweets de "+twitter_user_name)
    
      for page in Cursor(api.user_timeline,
                         screen_name=twitter_user_name, 
                         count=count_tweet).pages(page_limit):
        for tweet in page:
          parsed_tweet = {}
          parsed_tweet['user_id'] = tweet.user.id
          parsed_tweet['user_screen_name'] = twitter_user_name
          parsed_tweet['id_tweet'] = tweet.id
          parsed_tweet['created_at'] = tweet.created_at
          parsed_tweet['tweet'] = tweet.text
          parsed_tweet['retweet_count'] = tweet.retweet_count
          parsed_tweet['favorite_count'] = tweet.favorite_count
          parsed_tweet['liability'] = liability
          parsed_tweet['liability_label'] = liability_label[liability-1]
          parsed_tweet['count_followers'] = tweet.user.followers_count

          all_tweets.append(parsed_tweet)

      return all_tweets
    except Exception as e:
      print("Exception get_tweets pour "+twitter_user_name)
      return all_tweets

Récupération des tweets

In [ ]:
nb = 0
list_tweet = []

for site in data:
  list_tweet = list_tweet + get_tweets_from_user(site[2], site[1], api)
  nb += 1
  if (nb%10 == 0):
    print("Avancement : "+str(nb)+"/"+str(len(data)))

res = pd.DataFrame(list_tweet, columns=['user_id', 'user_screen_name', 'id_tweet', 'created_at', 'tweet', 'retweet_count', 'favorite_count', 'liability', 'liability_label', 'count_followers'])

Récupération des tweets de france_soir
Récupération des tweets de ThierryRegenere
Récupération des tweets de silvano_trotta
Exception get_tweets pour silvano_trotta
Récupération des tweets de emakrusi
Récupération des tweets de llp_le_vrai
Récupération des tweets de covid_infos
Récupération des tweets de cab2626
Récupération des tweets de lamentableFR
Récupération des tweets de Elishean
Récupération des tweets de DrMo7oG
Avancement : 10/425
Récupération des tweets de yetiblog
Récupération des tweets de lepolitique_fr
Récupération des tweets de les_repliques
Récupération des tweets de damocles_fr
Exception get_tweets pour damocles_fr
Récupération des tweets de jnewsfoot
Récupération des tweets de JournalElysee
Récupération des tweets de 20secondesfr
Exception get_tweets pour 20secondesfr
Récupération des tweets de Golfinvestigat1
Récupération des tweets de iiiprs
Récupération des tweets de voxdotcom
Avancement : 20/425
Récupération des tweets de usinenouvelle
Récupération des tweets de 

Vérification

In [ ]:
res

,user_id,user_screen_name,id_tweet,created_at,tweet,retweet_count,favorite_count,liability,liability_label,count_followers
0,78957336,france_soir,1507008907390062602,2022-03-24 14:58:17,"🎙️ ""Un Corse indépendantiste, c’est à la base ...",3,11,2,Site non fiable,126112
1,78957336,france_soir,1506984886061699072,2022-03-24 13:22:49,🍄 La start-up Mycophyto propose d’enrichir le...,8,31,2,Site non fiable,126112
2,78957336,france_soir,1506959719319093249,2022-03-24 11:42:49,📽 Dans un debriefing exclusif conduit avec Ari...,54,85,2,Site non fiable,126112
3,78957336,france_soir,1506958411115675650,2022-03-24 11:37:37,⚠️ Le conflit #UkraineRussie créant à la fois ...,21,36,2,Site non fiable,126112
4,78957336,france_soir,1506953912259121156,2022-03-24 11:19:45,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_...",213,350,2,Site non fiable,126112
...,...,...,...,...,...,...,...,...,...,...
997975,14354304,wikiHow,925845110461534209,2017-11-01 22:00:50,A little bit of gentle heat from a hair dryer ...,0,4,4,Site réputé fiable,73816
997976,14354304,wikiHow,925843158973366272,2017-11-01 21:53:05,@pxkelley @hiringourheroes Go out there and ge...,0,0,4,Site réputé fiable,73816
997977,14354304,wikiHow,925814991772676096,2017-11-01 20:01:09,#wikitips https://t.co/58AmUzhsaj https://t.co...,0,2,4,Site réputé fiable,73816
997978,14354304,wikiHow,925784921842634753,2017-11-01 18:01:40,If your dishwasher still holds water when you ...,1,3,4,Site réputé fiable,73816


Sauvegarde des tweets

In [ ]:
res.to_pickle(DATA_PATH+"data_tweets.pkl")

#  Section inutile étant donné la résolution du problème qu'elle traitait

Auparavant suite a certaines erreurs la récupération des tweets des 425 utilisateurs retenus saturait la RAM. Une autre solution pour récupérer les tweets a depuis été implémenté et, maintenant, il n'y a plus de problème de saturation et donc plus la nécessité d'utiliser cette partie du programme.

Création d'une classe TwitterUser contenant les fonctions de récupération de données (utilisateurs, tweets, ...)

In [ ]:
"""class TwitterUser():
  _screen_name = ""
  _id = ""
  _user_data = None
  followers = []
  following = []
  tweet = []
  like = []
  request_count = 0

  def __init__(self, sname = "", _json = {}):
    if (sname != ""):
      self._screen_name = sname
      self.get_info()
      if (_json != {}):
        self._json = _json

  def get_info(self):
    try:
      self._user_data = api.get_user(screen_name=self._screen_name)
    except Exception as e:
      print("Exception get_info")

  def get_all_data(self):
    self.get_tweets()
    # self.get_followers()
    # self.get_following()
    # self.get_like()

  def set_name(self, name):
    self._screen_name = name

  def set_id(self, _id):
    self._id = _id

  def get_followers(self):
    with tqdm(total=self._user_data.followers_count, desc="get_followers for " + self._user_data.screen_name) as pbar:
      for page in tweepy.Cursor(api.followers, screen_name=self._screen_name, count=200).pages():
        self.followers.extend(page)
        pbar.update(len(page))


  def get_following(self):
    with tqdm(total=self._user_data.friends_count, desc="get_following for " + self._user_data.screen_name) as pbar:
      for page in tweepy.Cursor(api.friends, screen_name=self._screen_name, count=200).pages():
        self.following.extend(page)
        pbar.update(len(page))

  # on ne peut prendre que les 1000 derniers
  def get_tweets(self):
    try:
      with tqdm(total=1000, desc="get_tweets for " + self._user_data.screen_name) as pbar:
        for page in tweepy.Cursor(api.user_timeline, screen_name=self._screen_name, count=200).pages():
          self.tweet.extend(page)
          pbar.update(len(page))
    except Exception as e:
      print("Exception get_tweets")

  # on ne peut prendre que les 3000 derniers
  def get_like(self):
    if self._user_data.protected:
      print("can get like from " + self._user_data.screen_name + ", the account is protected")
    else:
      with tqdm(total=3000, desc="get_likes for " + self._user_data.screen_name) as pbar:
        for page in tweepy.Cursor(api.favorites, screen_name=self._screen_name, count=200).pages():
          self.like.extend(page)
          pbar.update(len(page))"""

Récupération des tweets

In [ ]:
"""list_tweet = []

for site in data:
  user = TwitterUser(sname= site[2])
  user.get_all_data()

for tweet in user.tweet:
  list_tweet = list_tweet + [tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]]"""

Division de la liste des site en plusieurs sous-listes pour ne pas exéder la RAM alloué par Google Colab 

In [ ]:
# def split_list(a_list):
#     half = len(a_list)//2
#     return a_list[:half], a_list[half:]

# dataA, dataB = split_list(data)
# data1, data2 = split_list(dataA)
# data3, data4 = split_list(dataB)

Autre manière de séparer en plusieurs sous-listes : séparation basée sur le niveau de fiabilité des sites.

In [ ]:
"""data1 = [site for site in data if site[1] == 1]
data2 = [site for site in data if site[1] == 2]
data3 = [site for site in data if site[1] == 3]
data4 = [site for site in data if site[1] == 4]"""

'data1 = [site for site in data if site[1] == 1]\ndata2 = [site for site in data if site[1] == 2]\ndata3 = [site for site in data if site[1] == 3]\ndata4 = [site for site in data if site[1] == 4]'

Ajout de tous les tweets dans 4 tableaux qui vont contenir les données importantes de chaque tweet. Chaque tableau contient un quart des données de ***data***. Les tableaux sont ensuite sauvegardé au format pkl sur le Drive

Pour obtenir chaque morceau il faut relancer la session (ce qui libère la RAM) et exécuter le bloc de commande contenant le morceau voulu

Création et sauvegarde du tableau 1

Tentative de supprimer le problème de sur-copie des tweets (tweets du dernier utilisateur apparaissent 1 fois, tweets de l'avant dernier 2 fois, encore avant 3 fois, ...)

In [ ]:
"""list_tweet = []

for site in data1:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

res = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
res.to_pickle(DATA_PATH+"quartier1.pkl")"""

'list_tweet = []\n\nfor site in data1:\n  user = TwitterUser(sname= site[2])\n  user.get_all_data()\n  for tweet in user.tweet:\n    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])\n\nres = pd.DataFrame(list_tweet, columns=[\'user_id\', \'id_tweet\', \'tweet\', \'retweet_count\', \'favorite_count\', \'label_liability\'])\nres.to_pickle(DATA_PATH+"quartier1.pkl")'

In [ ]:
"""len(list_tweet)"""

'len(list_tweet)'

Création et sauvegarde du tableau 2

In [ ]:
"""list_tweet = []

for site in data2:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

res = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
res.to_pickle(DATA_PATH+"quartier2.pkl")"""

'list_tweet = []\n\nfor site in data2:\n  user = TwitterUser(sname= site[2])\n  user.get_all_data()\n  for tweet in user.tweet:\n    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])\n\nres = pd.DataFrame(list_tweet, columns=[\'user_id\', \'id_tweet\', \'tweet\', \'retweet_count\', \'favorite_count\', \'label_liability\'])\nres.to_pickle(DATA_PATH+"quartier2.pkl")'

Création et sauvegarde du tableau 3

In [ ]:
"""list_tweet = []

for site in data2:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

res = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
res.to_pickle(DATA_PATH+"quartier3.pkl")"""

'list_tweet = []\n\nfor site in data2:\n  user = TwitterUser(sname= site[2])\n  user.get_all_data()\n  for tweet in user.tweet:\n    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])\n\nres = pd.DataFrame(list_tweet, columns=[\'user_id\', \'id_tweet\', \'tweet\', \'retweet_count\', \'favorite_count\', \'label_liability\'])\nres.to_pickle(DATA_PATH+"quartier3.pkl")'

Création et sauvegarde du tableau 4

In [ ]:
"""list_tweet = []

for site in data2:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

res = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
res.to_pickle(DATA_PATH+"quartier4.pkl")"""

'list_tweet = []\n\nfor site in data2:\n  user = TwitterUser(sname= site[2])\n  user.get_all_data()\n  for tweet in user.tweet:\n    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])\n\nres = pd.DataFrame(list_tweet, columns=[\'user_id\', \'id_tweet\', \'tweet\', \'retweet_count\', \'favorite_count\', \'label_liability\'])\nres.to_pickle(DATA_PATH+"quartier4.pkl")'

Après avoir obtenu les 4 tableaux en ayant relancé 4 sessions, une pour la création de chaque tableau, on récupère les 4 tableaux

In [ ]:
"""tableauTweets1 = pd.read_pickle(DATA_PATH+"quartier1.pkl")
tableauTweets2 = pd.read_pickle(DATA_PATH+"quartier2.pkl")
tableauTweets3 = pd.read_pickle(DATA_PATH+"quartier3.pkl")
tableauTweets4 = pd.read_pickle(DATA_PATH+"quartier4.pkl")"""

'tableauTweets1 = pd.read_pickle(DATA_PATH+"quartier1.pkl")\ntableauTweets2 = pd.read_pickle(DATA_PATH+"quartier2.pkl")\ntableauTweets3 = pd.read_pickle(DATA_PATH+"quartier3.pkl")\ntableauTweets4 = pd.read_pickle(DATA_PATH+"quartier4.pkl")'

Concaténation des 4 tableaux en un seul et sauvegarde dans un seul fichier

In [ ]:
"""tableauTweets = pd.concat([tableauTweets1,tableauTweets2,tableauTweets3,tableauTweets4])
tableauTweets.reset_index(drop=True)
tableauTweets.to_pickle(DATA_PATH+"tableauTweets.pkl")"""

'tableauTweets = pd.concat([tableauTweets1,tableauTweets2,tableauTweets3,tableauTweets4])\ntableauTweets.reset_index(drop=True)\ntableauTweets.to_pickle(DATA_PATH+"tableauTweets.pkl")'

Réinitialisation du fichier PauchBas.pickle

In [ ]:
"""import pickle

with open(DATA_PATH+"PaucBas.pickle", "wb") as obj_file:
    pickle.dump([], obj_file, -1)"""

'import pickle\n\nwith open(DATA_PATH+"PaucBas.pickle", "wb") as obj_file:\n    pickle.dump([], obj_file, -1)'

Sauvegarde des données reçu pour chaque utilisateur Twitter recherché

In [ ]:
# with open(DATA_PATH+"PaucBas.pickle", "ab") as obj_file:
#   for site in data1:
#     user = TwitterUser(sname= site[2])
#     user.get_all_data()
#     pickle.dump([user], obj_file, pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(DATA_PATH+"PaucBas.pickle", "ab") as obj_file:
#   for site in data2:
#     user = TwitterUser(sname= site[2])
#     user.get_all_data()
#     pickle.dump([user], obj_file, pickle.HIGHEST_PROTOCOL)